In [1]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import rasterio.features
import rioxarray as rio
import xrspatial.multispectral as ms

# Import Planetary Computer tools
import pystac_client
import planetary_computer as pc
import odc
from odc.stac import stac_load
from odc.algo import to_rgba

In [2]:
crop_presence_data = pd.read_csv("Crop_Location_Data-20221201.csv")

In [3]:
crop_presence_data["Latitude"] = crop_presence_data["Latitude and Longitude"].apply(lambda x: float(x[1:-1].split(",")[0]))
crop_presence_data["Longitude"] = crop_presence_data["Latitude and Longitude"].apply(lambda x: float(x[1:-1].split(",")[1]))

In [4]:
crop_presence_data.head()

,Latitude and Longitude,Class of Land,Latitude,Longitude
0,"(10.323727047081501, 105.2516346045924)",Rice,10.323727,105.251635
1,"(10.322364360592521, 105.27843410554115)",Rice,10.322364,105.278434
2,"(10.321455902933202, 105.25254306225168)",Rice,10.321456,105.252543
3,"(10.324181275911162, 105.25118037576274)",Rice,10.324181,105.251180
4,"(10.324635504740822, 105.27389181724476)",Rice,10.324636,105.273892


In [5]:
box_size_deg = 0.10
resolution = 20  # meters per pixel 
scale = resolution / 111320.0 # degrees per pixel for CRS:4326 
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

In [6]:
def get_data_latlong(lat, long, time_window = "2020-03-20/2020-03-21"):
    min_lon = long-box_size_deg/2
    min_lat = lat-box_size_deg/2
    max_lon = long+box_size_deg/2
    max_lat = lat+box_size_deg/2
    bounds = (min_lon, min_lat, max_lon, max_lat)
    search = stac.search(collections=["sentinel-2-l2a"], bbox=bounds, datetime=time_window)
    items = list(search.get_all_items())
    xx = stac_load(
        items,
        bands=["red", "green", "blue", "nir", "SCL"],
        crs="EPSG:4326", # Latitude-Longitude
        resolution=scale, # Degrees
        chunks={"x": 2048, "y": 2048},
        dtype="uint16",
        patch_url=pc.sign,
        bbox=bounds
    )
    return xx

In [7]:
xx_lst = [0]*len(crop_presence_data)
lat_arr = crop_presence_data["Latitude"]
long_arr = crop_presence_data["Longitude"]
for inx in range(len(crop_presence_data)):
    xx = get_data_latlong(lat_arr[inx], long_arr[inx])
    xx_lst[inx] = xx
    print(inx)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
xx_lst_np_1 = [0] * 200
for inx in range(200):
    temp = xx_lst[inx].isel(time=0)[["red", "green", "blue"]].to_array()
    xx_lst_np_1[inx] = temp.data.compute()
    print(inx)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [9]:
xx_lst_np_2 = [0] * 200
for inx in range(200):
    temp = xx_lst[200+inx].isel(time=0)[["red", "green", "blue"]].to_array()
    xx_lst_np_2[inx] = temp.data.compute()
    print(200+inx)

200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399


In [10]:
xx_lst_np_3 = [0] * 200
for inx in range(200):
    temp = xx_lst[400+inx].isel(time=0)[["red", "green", "blue"]].to_array()
    xx_lst_np_3[inx] = temp.data.compute()
    print(400+inx)

400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599


In [11]:
crop_presence_data["Class"] = crop_presence_data["Class of Land"].apply(lambda x: 1 if x == "Rice" else 0)

In [12]:
for inx in range(200):
    xx_lst_np_1[inx] = np.resize(xx_lst_np_1[inx], (3, 557, 557)).reshape(3, 557*557)
    xx_lst_np_2[inx] = np.resize(xx_lst_np_2[inx], (3, 557, 557)).reshape(3, 557*557)
    xx_lst_np_3[inx] = np.resize(xx_lst_np_3[inx], (3, 557, 557)).reshape(3, 557*557)

In [13]:
for inx in range(200):
    xx_lst_np_1[inx] = xx_lst_np_1[inx].reshape(3*557*557)
    xx_lst_np_2[inx] = xx_lst_np_2[inx].reshape(3*557*557)
    xx_lst_np_3[inx] = xx_lst_np_3[inx].reshape(3*557*557)

In [14]:
x = np.concatenate([np.array(xx_lst_np_1), np.array(xx_lst_np_2), np.array(xx_lst_np_3)], axis = 0)

In [15]:
y = np.array(crop_presence_data["Class"])

In [16]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 101)

In [17]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()

In [18]:
clf.fit(x_train, y_train)

DecisionTreeClassifier()

In [19]:
pred = clf.predict(x_test)

In [20]:
from sklearn.metrics import classification_report

In [21]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.91      0.94      0.93        86
           1       0.95      0.91      0.93        94

    accuracy                           0.93       180
   macro avg       0.93      0.93      0.93       180
weighted avg       0.93      0.93      0.93       180



In [22]:
test_data = pd.read_csv("challenge_1_submission_template_correct_columns_fixed.csv")

In [23]:
test_data.head()

,id,target
0,"(10.18019073690894, 105.32022315786804)",NaN
1,"(10.561107033461816, 105.12772097986661)",NaN
2,"(10.623790611954897, 105.13771401411867)",NaN
3,"(10.583364246115156, 105.23946127195805)",NaN
4,"(10.20744446668854, 105.26844107128906)",NaN


In [24]:
test_data["Latitude"] = test_data["id"].apply(lambda x: float(x[1:-1].split(",")[0]))
test_data["Longitude"] = test_data["id"].apply(lambda x: float(x[1:-1].split(",")[1]))

In [25]:
test_data.head()

,id,target,Latitude,Longitude
0,"(10.18019073690894, 105.32022315786804)",NaN,10.180191,105.320223
1,"(10.561107033461816, 105.12772097986661)",NaN,10.561107,105.127721
2,"(10.623790611954897, 105.13771401411867)",NaN,10.623791,105.137714
3,"(10.583364246115156, 105.23946127195805)",NaN,10.583364,105.239461
4,"(10.20744446668854, 105.26844107128906)",NaN,10.207444,105.268441


In [ ]:
test_lst = [0]*250
lat_arr_test = test_data["Latitude"]
long_arr_test = test_data["Longitude"]
for inx in range(250):
    xx = get_data_latlong(lat_arr_test[inx], long_arr_test[inx])
    test_lst[inx] = xx
    print(inx)

In [29]:
test_lst_np_1 = [0] * 250
for inx in range(250):
    temp = test_lst[inx].isel(time=0)[["red", "green", "blue"]].to_array()
    test_lst_np_1[inx] = temp.data.compute()
    print(inx)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124


In [ ]:
test_lst_np_2 = [0] * 250
for inx in range(250):
    temp = test_lst[125+inx].isel(time=0)[["red", "green", "blue"]].to_array()
    test_lst_np_2[inx] = temp.data.compute()
    print(inx)

In [90]:
"""
test_lst_np_1 = test_lst_np_1[0:125]
test_lst_np_2 = test_lst_np_2[0:125]
"""

In [91]:
for inx in range(125):
    test_lst_np_1[inx] = np.resize(test_lst_np_1[inx], (3, 557, 557)).reshape(3, 557*557)
    test_lst_np_2[inx] = np.resize(test_lst_np_2[inx], (3, 557, 557)).reshape(3, 557*557)

In [92]:
for inx in range(125):
    test_lst_np_1[inx] = test_lst_np_1[inx].reshape(3*557*557)
    test_lst_np_2[inx] = test_lst_np_2[inx].reshape(3*557*557)

In [93]:
my_test = np.concatenate([np.array(test_lst_np_1), np.array(test_lst_np_2)], axis = 0)

In [95]:
pred_test = clf.predict(my_test)

In [97]:
len(pred_test)

250

In [98]:
test_data["target"] = pd.DataFrame({"target": pred_test})

In [99]:
test_data.head()

,id,target,Latitude,Longitude
0,"(10.18019073690894, 105.32022315786804)",1,10.180191,105.320223
1,"(10.561107033461816, 105.12772097986661)",1,10.561107,105.127721
2,"(10.623790611954897, 105.13771401411867)",1,10.623791,105.137714
3,"(10.583364246115156, 105.23946127195805)",1,10.583364,105.239461
4,"(10.20744446668854, 105.26844107128906)",1,10.207444,105.268441


In [101]:
export = test_data[["id", "target"]]

In [103]:
export["target"] = export["target"].apply(lambda x: "Rice" if 1 else "Non Rice")

In [104]:
export.to_csv("Test_result.csv", index = False)